# DS 2010 Final Project

## Import Packages

In [3]:
import pandas as pd
import math
import numpy as np
import datetime as dt

## Import Data

In [4]:
covid_19_data = pd.read_csv("Raw_Data/all-states-history-correct-range.csv")

anxiety_depression_data = pd.read_csv("Raw_Data/Indicators_of_Anxiety_or_Depression_Based_on_Reported_Frequency_of_Symptoms_During_Last_7_Days.csv")
# Starts Apr 23, Ends Nov 9

trips_data = pd.read_csv("Raw_Data/Trips_by_Distance.csv")
# trips_data2 = pd.read_csv("Raw_Data/Trips_by_Distance2.csv")

In [6]:
covid_19_data

,date,state,dataQualityGrade,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2020-11-21,AK,A,102.0,102.0,1,NaN,612.0,612.0,129.0,...,918745.0,14033,NaN,NaN,NaN,NaN,NaN,0,918745.0,14033
1,2020-11-21,AL,A,3457.0,3153.0,6,304.0,23449.0,23449.0,1300.0,...,1518186.0,9974,NaN,NaN,69591.0,NaN,NaN,0,1518186.0,9974
2,2020-11-21,AR,A+,2337.0,2141.0,16,196.0,8415.0,8415.0,922.0,...,1580941.0,15068,NaN,21856.0,NaN,116069.0,NaN,0,1580941.0,15068
3,2020-11-21,AS,D,0.0,NaN,0,NaN,NaN,NaN,NaN,...,1988.0,0,NaN,NaN,NaN,NaN,NaN,0,1988.0,0
4,2020-11-21,AZ,A+,6457.0,5985.0,30,472.0,24181.0,24181.0,1916.0,...,2083643.0,19754,355879.0,NaN,NaN,NaN,2083643.0,19754,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14780,2020-01-24,WA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,...,0.0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
14781,2020-01-23,MA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,...,2.0,1,NaN,NaN,NaN,NaN,NaN,0,2.0,1
14782,2020-01-23,WA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,...,0.0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
14783,2020-01-22,MA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,...,1.0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,1


In [9]:
anxiety_depression_data

,Phase,Indicator,Group,State,Subgroup,Time Period,Time Period Label,Value,Low CI,High CI,Confidence Interval,Quartile range
0,1,Symptoms of Depressive Disorder,National Estimate,United States,United States,1,Apr 23 - May 5,23.5,22.7,24.3,22.7 - 24.3,NaN
1,1,Symptoms of Depressive Disorder,By Age,United States,18 - 29 years,1,Apr 23 - May 5,32.7,30.2,35.2,30.2 - 35.2,NaN
2,1,Symptoms of Depressive Disorder,By Age,United States,30 - 39 years,1,Apr 23 - May 5,25.7,24.1,27.3,24.1 - 27.3,NaN
3,1,Symptoms of Depressive Disorder,By Age,United States,40 - 49 years,1,Apr 23 - May 5,24.8,23.3,26.2,23.3 - 26.2,NaN
4,1,Symptoms of Depressive Disorder,By Age,United States,50 - 59 years,1,Apr 23 - May 5,23.2,21.5,25.0,21.5 - 25.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3832,3,Symptoms of Anxiety Disorder or Depressive Dis...,By State,Virginia,Virginia,18,Oct 28 - Nov 9,36.7,32.7,40.9,32.7 - 40.9,25.5-38.8
3833,3,Symptoms of Anxiety Disorder or Depressive Dis...,By State,Washington,Washington,18,Oct 28 - Nov 9,38.8,35.3,42.3,35.3 - 42.3,25.5-38.8
3834,3,Symptoms of Anxiety Disorder or Depressive Dis...,By State,West Virginia,West Virginia,18,Oct 28 - Nov 9,39.3,32.4,46.5,32.4 - 46.5,38.9-41.6
3835,3,Symptoms of Anxiety Disorder or Depressive Dis...,By State,Wisconsin,Wisconsin,18,Oct 28 - Nov 9,39.2,34.2,44.4,34.2 - 44.4,38.9-41.6


In [58]:
trips_data = pd.concat([trips_data, trips_data2])
trips_data


,Level,Date,State FIPS,State Postal Code,County FIPS,County Name,Population Staying at Home,Population Not Staying at Home,Number of Trips,Number of Trips <1,Number of Trips 1-3,Number of Trips 3-5,Number of Trips 5-10,Number of Trips 10-25,Number of Trips 25-50,Number of Trips 50-100,Number of Trips 100-250,Number of Trips 250-500,Number of Trips >=500
0,County,2020/04/21,1.0,AL,1005.0,Barbour County,3710.0,21171.0,84775.0,17881.0,17606.0,12338.0,15686.0,12859.0,5581.0,2399.0,326.0,92.0,7.0
1,County,2020/04/21,1.0,AL,1007.0,Bibb County,3988.0,18412.0,76633.0,18209.0,16115.0,10313.0,10157.0,11346.0,7691.0,2164.0,261.0,377.0,0.0
2,County,2020/04/21,1.0,AL,1009.0,Blount County,8946.0,48894.0,177929.0,34448.0,34009.0,22149.0,28261.0,30849.0,20219.0,6349.0,1261.0,337.0,47.0
3,County,2020/04/21,1.0,AL,1011.0,Bullock County,1229.0,8909.0,41334.0,9394.0,6803.0,4123.0,8343.0,5915.0,4551.0,1853.0,323.0,0.0,29.0
4,County,2020/04/21,1.0,AL,1013.0,Butler County,3125.0,16555.0,59190.0,12262.0,13525.0,6772.0,8818.0,10032.0,3648.0,2860.0,1225.0,35.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6383,County,2020/11/09,1.0,AL,1005.0,Barbour County,5331.0,19550.0,72823.0,14202.0,17185.0,8736.0,14077.0,9532.0,5601.0,2857.0,560.0,61.0,12.0
6384,County,2020/11/09,1.0,AL,1003.0,Baldwin County,47350.0,170672.0,706492.0,135742.0,179316.0,97271.0,121624.0,121072.0,36738.0,9169.0,4165.0,1257.0,138.0
6385,County,2020/11/09,1.0,AL,1001.0,Autauga County,12007.0,43594.0,175811.0,37100.0,43756.0,26752.0,25085.0,31018.0,7964.0,2767.0,1101.0,251.0,17.0
6386,County,2020/11/09,2.0,AK,2164.0,Lake and Peninsula Borough,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Clean trip data

In [60]:
td = trips_data.groupby(by = ['Date', 'State Postal Code']).sum().drop(columns= ['State FIPS', 'County FIPS']).reset_index()
td['Week'] = np.floor(td.index / 51 / 7)
td2 = td.groupby(by = ['Week', 'State Postal Code']).sum().reset_index()
td2

,Week,State Postal Code,Population Staying at Home,Population Not Staying at Home,Number of Trips,Number of Trips <1,Number of Trips 1-3,Number of Trips 3-5,Number of Trips 5-10,Number of Trips 10-25,Number of Trips 25-50,Number of Trips 50-100,Number of Trips 100-250,Number of Trips 250-500,Number of Trips >=500
0,0.0,AK,3220673.0,7082591.0,2.897419e+07,8992772.0,8231687.0,3635131.0,3827901.0,2736117.0,1083549.0,316497.0,98020.0,15715.0,36802.0
1,0.0,AL,14561732.0,53868462.0,1.934874e+08,43540272.0,49001370.0,25808634.0,31077134.0,29209804.0,9846620.0,3411602.0,1273558.0,265330.0,53040.0
2,0.0,AR,9047480.0,33146070.0,1.194944e+08,27916370.0,31271648.0,15000080.0,18399048.0,17282118.0,6057438.0,2394556.0,950380.0,189272.0,33490.0
3,0.0,AZ,28810262.0,71592782.0,2.514977e+08,72521122.0,60440296.0,28613068.0,34635610.0,36573306.0,12008498.0,4024546.0,2123438.0,386488.0,171346.0
4,0.0,CA,161075465.0,392723165.0,1.275602e+09,340797705.0,335866619.0,144512367.0,174459323.0,176019070.0,68268533.0,24249859.0,8750564.0,1991002.0,686924.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1474,28.0,VT,3112011.0,8161371.0,3.335343e+07,6354524.0,8685732.0,4231276.0,4827171.0,5648798.0,2611117.0,724701.0,219875.0,40951.0,9283.0
1475,28.0,WA,40022575.0,95618063.0,3.629090e+08,86031010.0,90566383.0,42190273.0,55452475.0,57979032.0,21177510.0,6530974.0,2312758.0,401626.0,266979.0
1476,28.0,WI,29082135.0,75562089.0,3.055331e+08,65971931.0,80193353.0,37947144.0,46006069.0,45231781.0,17899922.0,8094189.0,3517971.0,550479.0,120279.0
1477,28.0,WV,9088279.0,23416697.0,9.196436e+07,18324623.0,24856511.0,11423885.0,13832736.0,13681066.0,5852528.0,2931675.0,911250.0,131684.0,18401.0


## Clean COVID-19 Data

In [3]:
# KEEP:
# DC: District of Columbia

# REMOVE:
# AS: American Samoa
# PR: Puerto Rico
# GU: Guam
# MP: Northern Mariana Islands
# VI: US Virgin Islands
remove_states = ['AS', 'PR', 'GU', 'MP', 'VI']

### prepare covid data

In [4]:
valuable_cols = ['state', 'date', 'death', 'deathConfirmed', 
                 'hospitalizedCurrently', 'positiveCasesViral', 
                 'positiveIncrease', 'totalTestsPeopleViral', 
                 'totalTestsPeopleViralIncrease', 'totalTestsViral']

In [5]:
# Remove undesired columns
covid_data = covid_19_data.copy()[valuable_cols]
# Convert string to datetime
covid_data['date'] = covid_data['date'].astype('datetime64[ns]')
# Convert target_ranges to list of 

### create Time Period column

In [6]:
ranges_list = ['4/23/2020-5/5/2020', '5/7/2020-5/12/2020', '5/14/2020-5/19/2020', '5/21/2020-5/26/2020',
               '5/28/2020-6/2/2020', '6/4/2020-6/9/2020', '6/11/2020-6/16/2020', '6/18/2020-6/23/2020',
               '6/25/2020-6/30/2020', '7/2/2020-7/7/2020', '7/9/2020-7/14/2020', '7/16/2020-7/21/2020',                  
               '8/19/2020-8/31/2020', '9/2/2020-9/14/2020', '9/16/2020-9/28/2020', '9/30/2020-10/12/2020', 
               '10/14/2020-10/26/2020', '10/28/2020-11/9/2020']

# Convert ranges_list to a list of lists of start and end datetimes
period_list = []
for r in ranges_list:
    r_list = []
    for date_str in r.split('-'):
        date = dt.datetime.strptime(date_str, '%m/%d/%Y').date()
        r_list.append(date)
    period_list.append(r_list)
    
# Create Time Period column
time_periods = [] # -1: before first period | -2: after last period | 0-n: respective period
for index, row in covid_data.iterrows():
    true_period = np.NaN
    if row['date'] < period_list[0][0]:
        true_period = -1
    elif row['date'] > period_list[-1][-1]:
        true_period = -2
    else:
        for period in period_list:
            if period[0] <= row['date'] <= period[1]:
                true_period = period_list.index(period) + 1
    time_periods.append(true_period)

# Add Time Period column to covid_data
covid_data['Time Period'] = time_periods

# Make a copy before cleaning
data = covid_data.copy()

# Remove rows with invalid time periods
data.drop(data.loc[data['Time Period'] == -1].index, inplace=True)
data.drop(data.loc[data['Time Period'] == -2].index, inplace=True)
data.drop(data.loc[data['Time Period'] == np.NaN].index, inplace=True)

In [7]:
states = list(set(data['state'].values))
final_states = [item for item in states if item not in remove_states]

### Fill NaN and groupby Time Period for each state

In [26]:
clean_data = []
for state in final_states:
    state_data = data.loc[data['state'] == state]
    print(state_data)
    exit()
    # TODO fill NaN by state
    state_data = state_data.groupby(['Time Period', 'state']).mean().reset_index()
    clean_data.append(state_data)
clean_df = pd.concat(clean_data)

      state       date   death  deathConfirmed  hospitalizedCurrently  \
430      OH 2020-04-23   656.0           618.0                    NaN   
486      OH 2020-04-24   690.0           649.0                    NaN   
542      OH 2020-04-25   711.0           671.0                    NaN   
598      OH 2020-04-26   728.0           687.0                    NaN   
654      OH 2020-04-27   753.0           712.0                    NaN   
...     ...        ...     ...             ...                    ...   
11406    OH 2020-11-05  5461.0          5133.0                 2075.0   
11462    OH 2020-11-06  5494.0          5165.0                 2170.0   
11518    OH 2020-11-07  5506.0          5177.0                 2188.0   
11574    OH 2020-11-08  5517.0          5186.0                 2286.0   
11630    OH 2020-11-09  5524.0          5193.0                 2533.0   

       positiveCasesViral  positiveIncrease  totalTestsPeopleViral  \
430               14142.0               577          

      state       date   death  deathConfirmed  hospitalizedCurrently  \
412      LA 2020-04-23  1540.0             NaN                 1727.0   
468      LA 2020-04-24  1601.0             NaN                 1697.0   
524      LA 2020-04-25  1644.0             NaN                 1700.0   
580      LA 2020-04-26  1670.0             NaN                 1701.0   
636      LA 2020-04-27  1697.0          1599.0                 1683.0   
...     ...        ...     ...             ...                    ...   
11388    LA 2020-11-05  5995.0          5766.0                  636.0   
11444    LA 2020-11-06  6016.0          5787.0                  644.0   
11500    LA 2020-11-07  6016.0          5787.0                  644.0   
11556    LA 2020-11-08  6036.0          5807.0                  622.0   
11612    LA 2020-11-09  6048.0          5819.0                  652.0   

       positiveCasesViral  positiveIncrease  totalTestsPeopleViral  \
412               25739.0               481          

      state       date  death  deathConfirmed  hospitalizedCurrently  \
425      NH 2020-04-23   48.0             NaN                   91.0   
481      NH 2020-04-24   51.0             NaN                   92.0   
537      NH 2020-04-25   53.0             NaN                   89.0   
593      NH 2020-04-26   60.0             NaN                   95.0   
649      NH 2020-04-27   60.0             NaN                  101.0   
...     ...        ...    ...             ...                    ...   
11401    NH 2020-11-05  486.0             NaN                   44.0   
11457    NH 2020-11-06  488.0             NaN                   48.0   
11513    NH 2020-11-07  489.0             NaN                   49.0   
11569    NH 2020-11-08  489.0             NaN                   55.0   
11625    NH 2020-11-09  489.0             NaN                   56.0   

       positiveCasesViral  positiveIncrease  totalTestsPeopleViral  \
425                   NaN                97                    Na

In [9]:
clean_df

,Time Period,state,death,deathConfirmed,hospitalizedCurrently,positiveCasesViral,positiveIncrease,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral
0,1.0,OH,884.692308,821.538462,1020.250000,16927.153846,527.076923,NaN,0.0,1.251098e+05
1,2.0,OH,1340.333333,1218.500000,1008.500000,22626.833333,612.333333,NaN,0.0,1.891108e+05
2,3.0,OH,1621.166667,1468.000000,917.000000,25992.833333,538.500000,NaN,0.0,2.539890e+05
3,4.0,OH,1937.000000,1743.333333,883.666667,29521.500000,595.000000,NaN,0.0,3.178662e+05
4,5.0,OH,2166.166667,1954.166667,781.333333,32822.000000,485.166667,NaN,0.0,3.831307e+05
...,...,...,...,...,...,...,...,...,...,...
13,14.0,NC,2933.538462,2933.538462,869.384615,178716.538462,1258.230769,NaN,0.0,2.479868e+06
14,15.0,NC,3304.153846,3297.923077,901.692308,195887.615385,1643.153846,NaN,0.0,2.838277e+06
15,16.0,NC,3673.769231,3642.307692,997.000000,215906.461538,1816.153846,NaN,0.0,3.215528e+06
16,17.0,NC,4010.230769,3959.000000,1167.846154,240750.153846,2097.000000,NaN,0.0,3.663452e+06


In [12]:
clean_df.to_csv("Clean_Data/clean_covid_data.csv", index= False)

## Clean Anxiety/Depression Data

In [45]:
data = anxiety_depression_data.copy()

In [46]:
clean_data = data.loc[data['State'] != 'United States']
clean_data = clean_data.drop(['Phase', 'Group', 'Subgroup', 'Time Period Label',
                             'Low CI', 'High CI', 'Confidence Interval', 'Quartile range'], axis=1)

In [51]:
clean_data
# 918 rows = 2754 rows / 3 Indicators

,Indicator,State,Time Period,Value
19,Symptoms of Depressive Disorder,Alabama,1,18.6
20,Symptoms of Depressive Disorder,Alaska,1,19.2
21,Symptoms of Depressive Disorder,Arizona,1,22.4
22,Symptoms of Depressive Disorder,Arkansas,1,26.6
23,Symptoms of Depressive Disorder,California,1,25.4
...,...,...,...,...
3832,Symptoms of Anxiety Disorder or Depressive Dis...,Virginia,18,36.7
3833,Symptoms of Anxiety Disorder or Depressive Dis...,Washington,18,38.8
3834,Symptoms of Anxiety Disorder or Depressive Dis...,West Virginia,18,39.3
3835,Symptoms of Anxiety Disorder or Depressive Dis...,Wisconsin,18,39.2


In [72]:
depression_data = clean_data.loc[clean_data['Indicator'] == 'Symptoms of Depressive Disorder']
anxiety_data = clean_data.loc[clean_data['Indicator'] == 'Symptoms of Anxiety Disorder']
both_data = clean_data.loc[clean_data['Indicator'] == 'Symptoms of Anxiety Disorder or Depressive Disorder']

In [73]:
merged_data = pd.merge(depression_data, anxiety_data, on=['State', 'Time Period'])
merged_data = pd.merge(merged_data, both_data, on=['State', 'Time Period'])

In [74]:
merged_data = merged_data.drop(['Indicator_x', 'Indicator_y', 'Indicator'], axis=1)
merged_data.columns = ['State', 'Period', 'Depression_Score', 'Anxiety_Score', 'Mix_Score']


In [75]:
merged_data

,State,Period,Depression_Score,Anxiety_Score,Mix_Score
0,Alabama,1,18.6,25.6,30.3
1,Alaska,1,19.2,27.7,31.5
2,Arizona,1,22.4,32.3,36.4
3,Arkansas,1,26.6,33.7,38.0
4,California,1,25.4,30.8,37.4
...,...,...,...,...,...
913,Virginia,18,24.0,30.5,36.7
914,Washington,18,24.4,34.7,38.8
915,West Virginia,18,25.7,35.7,39.3
916,Wisconsin,18,27.0,36.0,39.2


In [ ]:
clean_df.to_csv("Clean_Data/clean_label_data.csv", index= False)